In [2]:
import pandas as pd

In [3]:
# Loading the datasets
occupation_data = pd.read_excel('/Users/kiranmaireddy/Desktop/Semester 4/Result/mapped_occupational_data_with_algorithm_functions.xlsx')
algorithm_data = pd.read_excel('/Users/kiranmaireddy/Desktop/Semester 4/Result/mapped_algorithm_data_with_AF.xlsx')

In [4]:
# Converting algorithm functions columns to sets for easier comparison and matching
occupation_data['algorithm_functions'] = occupation_data['algorithm_functions'].apply(lambda x: eval(x) if isinstance(x, str) else [])
algorithm_data['algorithm_functions'] = algorithm_data['algorithm_functions'].apply(lambda x: eval(x) if isinstance(x, str) else [])

In [5]:
# Exploding the rows in algorithm data so each row represents a single algorithm function
algorithm_data_exploded = algorithm_data.explode('algorithm_functions')

In [6]:
# Performing a left join to match occupation data with algorithm data based on algorithm functions
merged_data = pd.merge(
    occupation_data.explode('algorithm_functions'),
    algorithm_data_exploded,
    on='algorithm_functions',
    suffixes=('_occupation', '_algorithm'),
    how='left'
)

In [7]:
# Counting the frequency of each algorithm for each algorithm function to compute relevance
relevance_scores = merged_data.groupby(['algorithm_functions', 'Algorithm ']).size().reset_index(name='relevance_score')

In [8]:
# Merging relevance scores back into merged data
merged_data = pd.merge(merged_data, relevance_scores, on=['algorithm_functions', 'Algorithm '], how='left')

In [9]:
# Setting a maximum number of algorithms to keep per occupation
max_algorithms = 3  

In [10]:
# Defining a function to select top algorithms based on relevance score
def select_top_algorithms(group):
    # Sorting algorithms by relevance score in descending order, then by algorithm name as a tiebreaker
    top_algorithms = group.sort_values(by='relevance_score', ascending=False).drop_duplicates(subset=['Algorithm ']).head(max_algorithms)
    return list(top_algorithms['Algorithm '])

In [11]:
# Applying groupby and aggregate with the custom function for algorithm selection
grouped_data = (
    merged_data.groupby(['Industry', 'Occupation', 'Code', 'Detailed Work Activities'])
    .apply(lambda x: pd.Series({
        'Combined Algorithm Functions': list(set(x['algorithm_functions'])),
        'Algorithms': select_top_algorithms(x)
    }))
    .reset_index()
)


/var/folders/j6/_1y5vk8x6k54363n4fvnsss40000gn/T/ipykernel_18232/203886865.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


In [12]:
# Saving the output
output_path = '/Users/kiranmaireddy/Desktop/Semester 4/Result/merged_occupational_and_algorithm_data_with_AF_.xlsx'
grouped_data.to_excel(output_path, index=False)

print("Merging complete. The relevance-based merged data is saved as 'merged_occupational_and_algorithm_data__with_AF_.xlsx'.")

Merging complete. The relevance-based merged data is saved as 'merged_occupational_and_algorithm_data__with_AF_.xlsx'.
